# Introduction

Detailed in this Jupyter Notebook is the methodology utilized to Pen Test Active Directory. This Jupyter Notebook will also serve to standardize techniques across lab resources and train resources on a common methodology.

## Setup

Jupyter Notebooks have a kernel that can be either Python 3 (allowing you to execute Python code directly in code cells) or a Bash kernel (enabling you to run Linux commands). 

Most Kali Linux commands are executed in a Bash terminal, and you can change the Jupyter Notebook kernel to Bash via Kernel > Change Kernel or using the option in the top-right corner.

However, in this case, we will use Python and its os.system('command') function to execute Kali tools. This approach allows us to:

>Easily integrate Jupyter Notebook with Kali Linux tools.

>Capture command outputs for documentation.

>Analyze results using Python’s data science capabilities.

>Later, enhance analysis with Anaconda and DeepSeek.

Make sure to run the setup scripts to install the tools before hand, or via the below cells.

## Setup All Tools 

In [ ]:
import os
os.system('echo "kali" | sudo -S apt update && sudo -S apt install -y nmap nbtscan')

## Install Tools via Jupiter Notebook

I am experimenting with the best way to install tools to work interactively within Jupyter Notebook. Here are some examples:

In [ ]:
!echo "kali" | sudo apt update && apt install -y nmap


In [ ]:
!echo 'kali' | sudo apt upgrade -y nmap

In [ ]:
import os
os.system('echo "kali" | sudo -S apt update && sudo -S apt install -y nmap')

In [ ]:
import os
os.system('echo "kali" | sudo -S apt update && apt upgrade -y nmap')

In [ ]:
!nmap --version

In [ ]:
import os
os.system("nmap --version")

In [ ]:
import subprocess

# Define the nmap command
command = ["nmap", "-sP", "10.2.10.0/24"]

# Run the command and capture the output
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Print the output
if result.returncode == 0:
    print("Nmap scan results:")
    print(result.stdout)
else:
    print("Error running nmap:")
    print(result.stderr)

In [ ]:
# In another cell, you can use the information from previous result

print(result)

# Reconnaissance


> Always save your results for later analysie, create libary for later use and name each text tile. Then use all of the outputs to creat a report.

> Once the variables have been setup, you can Runtime, Run All to create the report.

## START TIME

In [ ]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

#@ Defaults
Prior to running any cells assign a value to the variables and run the cell. This will change the targets for subdomain enumeration without needing to modify the script parameters. An example of an acceptable domain to replace the parameter with would be: "protiviti.com". Next set the ORG_NAME variable to the name of the target org.

In [ ]:
#Set DOMAIN & FOLDER_NAME Variable
DOMAIN = "example.com"
ORG_NAME = "example"
FOLDER_NAME = "example"
!mkdir $FOLDER_NAME

In [ ]:
print("The Following variables have been set:")
print("")
print("DOMAIN NAME: " + DOMAIN)
print("FOLDER NAME: " + FOLDER_NAME)
print("")

In [ ]:
print("Cloudenum in progress")
!python3 {configDict['cloudenum']['path']} -k $ORG_NAME -t 50 --disable-gcp --disable-azure -l cloudenum.csv
!mv --target-directory=$FOLDER_NAME/ cloudenum.csv
print("")
print("Cloudenum Complete")

## Network Discovery

In [ ]:
!ip a

In [2]:
!for i in {1..255}; do ping -c 1 10.2.10.$i | grep "bytes from" | cut -d " " -f 4 | tr -d ":" & done

10.2.10.11
10.2.10.21
10.2.10.254


In [1]:
# Create Targets.txt
!for i in {1..255}; do ping -c 1 10.2.10.$i | grep "bytes from" | cut -d " " -f 4 | tr -d ":" & done > targets.txt

In [4]:
# Read and print the contents of targets.txt
with open('targets.txt', 'r') as file:
    targets = file.read()
print(targets)

10.2.10.11
10.2.10.21
10.2.10.254



In [3]:
!nmap -n -sn 10.2.10.0/24 -oG - | awk '/Up$/{print $2}'

10.2.10.11
10.2.10.21
10.2.10.254


In [ ]:
# Scan for Open Ports (using nmap):

!nmap -sP 10.2.10.0/24
!nmap -p- -sCV -sS -T4 10.2.10.11 -vv
!nmap -n -sn 10.2.10.0/24 -oG - | awk '/Up$/{print $2}'
!nmap -iL targets.txt -sV --open -oG - | grep "Ports:"

In [5]:
!nmap -sP 10.2.10.0/24

Starting Nmap 7.95 ( https://nmap.org ) at 2025-03-05 18:56 EST
Nmap scan report for 10.2.10.11
Host is up (0.0015s latency).
Nmap scan report for 10.2.10.21
Host is up (0.0015s latency).
Nmap scan report for 10.2.10.254
Host is up (0.00084s latency).
Nmap done: 256 IP addresses (3 hosts up) scanned in 4.53 seconds


In [6]:
!nmap -iL targets.txt -sV --open -oG - | grep "Ports:"

Host: 10.2.10.11 ()	Ports: 53/open/tcp//domain//Simple DNS Plus/, 88/open/tcp//kerberos-sec//Microsoft Windows Kerberos (server time: 2025-03-06 14:56:47Z)/, 135/open/tcp//msrpc//Microsoft Windows RPC/, 139/open/tcp//netbios-ssn//Microsoft Windows netbios-ssn/, 389/open/tcp//ldap//Microsoft Windows Active Directory LDAP (Domain: ludus.domain0., Site: Default-First-Site-Name)/, 445/open/tcp//microsoft-ds?///, 464/open/tcp//kpasswd5?///, 593/open/tcp//ncacn_http//Microsoft Windows RPC over HTTP 1.0/, 636/open/tcp//tcpwrapped///, 3268/open/tcp//ldap//Microsoft Windows Active Directory LDAP (Domain: ludus.domain0., Site: Default-First-Site-Name)/, 3269/open/tcp//tcpwrapped///, 3389/open/tcp//ms-wbt-server//Microsoft Terminal Services/, 5986/open/tcp//ssl|http//Microsoft HTTPAPI httpd 2.0 (SSDP|UPnP)/	Ignored State: filtered (987)
Host: 10.2.10.21 ()	Ports: 135/open/tcp//msrpc//Microsoft Windows RPC/, 445/open/tcp//microsoft-ds?///, 3389/open/tcp//ms-wbt-server//Microsoft Terminal Services/

In [7]:
!nmap -sP -iL targets.txt

Starting Nmap 7.95 ( https://nmap.org ) at 2025-03-05 18:57 EST
Nmap scan report for 10.2.10.11
Host is up (0.00052s latency).
Nmap scan report for 10.2.10.21
Host is up (0.00054s latency).
Nmap scan report for 10.2.10.254
Host is up (0.00028s latency).
Nmap done: 3 IP addresses (3 hosts up) scanned in 0.09 seconds


In [8]:
!nmap -p- -sCV -sS -T4 10.2.10.11 -vv

Starting Nmap 7.95 ( https://nmap.org ) at 2025-03-05 18:57 EST
NSE: Loaded 157 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 18:57
Completed NSE at 18:57, 0.00s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 18:57
Completed NSE at 18:57, 0.00s elapsed
NSE: Starting runlevel 3 (of 3) scan.
Initiating NSE at 18:57
Completed NSE at 18:57, 0.00s elapsed
Initiating Ping Scan at 18:57
Scanning 10.2.10.11 [4 ports]
Completed Ping Scan at 18:57, 0.02s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 18:57
Completed Parallel DNS resolution of 1 host. at 18:57, 0.01s elapsed
Initiating SYN Stealth Scan at 18:57
Scanning 10.2.10.11 [65535 ports]
Discovered open port 139/tcp on 10.2.10.11
Discovered open port 3389/tcp on 10.2.10.11
Discovered open port 135/tcp on 10.2.10.11
Discovered open port 53/tcp on 10.2.10.11
Discovered open port 445/tcp on 10.2.10.11


## Netbios Enumeration

In [10]:
!nbtscan -f targets.txt

Doing NBT name scan for addresses from targets.txt

IP address       NetBIOS Name     Server    User             MAC address      
------------------------------------------------------------------------------


In [15]:
!nbtscan -r 10.2.10.0/24
!nbtscan -v -s : 10.2.10.0/24

Doing NBT name scan for addresses from 10.2.10.0/24

IP address       NetBIOS Name     Server    User             MAC address      
------------------------------------------------------------------------------


In [12]:
!nmap -iL targets.txt -sU -p 137 --script nbstat.nse

Starting Nmap 7.95 ( https://nmap.org ) at 2025-03-05 18:58 EST
Nmap scan report for 10.2.10.11
Host is up (0.0024s latency).

PORT    STATE         SERVICE
137/udp open|filtered netbios-ns

Nmap scan report for 10.2.10.21
Host is up (0.0027s latency).

PORT    STATE         SERVICE
137/udp open|filtered netbios-ns

Nmap scan report for 10.2.10.254
Host is up (0.00062s latency).

PORT    STATE  SERVICE
137/udp closed netbios-ns

Nmap done: 3 IP addresses (3 hosts up) scanned in 4.44 seconds


In [13]:
!crackmapexec smb 10.2.10.0/24 

SMB         10.2.10.11      445    SA-DC01-2022     [*] Windows Server 2022 Build 20348 x64 (name:SA-DC01-2022) (domain:ludus.domain) (signing:True) (SMBv1:False)
SMB         10.2.10.21      445    SA-WIN11-22H2-1  [*] Windows 11 Build 22621 x64 (name:SA-WIN11-22H2-1) (domain:ludus.domain) (signing:False) (SMBv1:False)


## SMB Enumeration


In [16]:
# List SMB Shares (using smbclient):

!smbclient -L //10.2.10.11 -N

Anonymous login successful

	Sharename       Type      Comment
	---------       ----      -------
Reconnecting with SMB1 for workgroup listing.
do_connect: Connection to 10.2.10.11 failed (Error NT_STATUS_RESOURCE_NAME_NOT_FOUND)
Unable to connect with SMB1 -- no workgroup available


In [17]:
# Check for Null Session Vulnerability (using rpcclient):

!rpcclient -U administrator 10.2.10.11
!rpcclient -U "" 10.2.10.11



Password for [WORKGROUP\administrator]:Password for [WORKGROUP\]:

In [19]:
# Enumerate SMB Users (using enum4linux):

#!enum4linux -U 10.2.10.11
!enum4linux-ng -A 10.2.10.11

ENUM4LINUX - next generation (v1.3.4)

|    Target Information    |
[*] Target ........... 10.2.10.11
[*] Username ......... ''
[*] Random Username .. 'smyzdhcq'
[*] Password ......... ''
[*] Timeout .......... 5 second(s)

|    Listener Scan on 10.2.10.11    |
[*] Checking LDAP
[+] LDAP is accessible on 389/tcp
[*] Checking LDAPS
[+] LDAPS is accessible on 636/tcp
[*] Checking SMB
[+] SMB is accessible on 445/tcp
[*] Checking SMB over NetBIOS
[+] SMB over NetBIOS is accessible on 139/tcp

|    Domain Information via LDAP for 10.2.10.11    |
[*] Trying LDAP
[+] Appears to be root/parent DC
[+] Long domain name is: ludus.domain

|    NetBIOS Names and Workgroup/Domain for 10.2.10.11    |
[-] Could not get NetBIOS names information via 'nmblookup': timed out

|    SMB Dialect Check on 10.2.10.11    |
[*] Trying on 445/tcp
[+] Supported dialects and settings:
Supported dialects:
  SMB 1.0: false
  SMB 2.02: true
  SMB 2.1: true
  SMB 3.0: true
  SMB 3.1.1: true
Preferred dialect: SMB 3.0


## AD Username and password list from Website

## Create Username list from LinkedIn


https://github.com/initstring/linkedin2username

linkedin2username.py -c targetco -d 5 -n 'targetco.com'


usage: linkedin2username.py [-h] -c COMPANY [-n DOMAIN] [-d DEPTH]
  [-s SLEEP] [-x PROXY] [-k KEYWORDS] [-g] [-o OUTPUT]

OSINT tool to generate lists of probable usernames from a given company's LinkedIn page.
This tool may break when LinkedIn changes their site.
Please open issues on GitHub to report any inconsistencies.

optional arguments:
  -h, --help            show this help message and exit
  -c COMPANY, --company COMPANY
                        Company name exactly as typed in the company linkedin profile page URL.
  -n DOMAIN, --domain DOMAIN
                        Append a domain name to username output.
                        [example: "-n targetco.com" would output jschmoe@targetco.com]
  -d DEPTH, --depth DEPTH
                        Search depth (how many loops of 25). If unset, will try to grab them
                        all.
  -s SLEEP, --sleep SLEEP
                        Seconds to sleep between search loops. Defaults to 0.
  -x PROXY, --proxy PROXY
                        Proxy server to use. WARNING: WILL DISABLE SSL VERIFICATION.
                        [example: "-p https://localhost:8080"]
  -k KEYWORDS, --keywords KEYWORDS
                        Filter results by a a list of command separated keywords.
                        Will do a separate loop for each keyword,
                        potentially bypassing the 1,000 record limit.
                        [example: "-k 'sales,human resources,information technology']
  -g, --geoblast        Attempts to bypass the 1,000 record search limit by running
                        multiple searches split across geographic regions.
  -o OUTPUT, --output OUTPUT
                        Output Directory, defaults to li2u-output

## LDAP Enumeration

In [22]:
ldapsearch -x -h 10.2.10.11 -b "DC=domain,DC=com" -s sub "(objectClass=*)" 

SyntaxError: invalid syntax (4126407748.py, line 1)

In [21]:
# Extract Users from LDAP:

ldapsearch -x -h 192.168.1.1 -b "DC=domain,DC=com" "(objectClass=user)" sAMAccountName #> users.txt

SyntaxError: invalid syntax (701938972.py, line 3)

In [ ]:
# Extract Computers from LDAP:

ldapsearch -x -h 192.168.1.1 -b "DC=domain,DC=com" "(objectClass=computer)" name #> computers.txt


## Kerberos Enumeration

In [ ]:
# Enumerate Kerberos Users (using kerbrute):

kerbrute userenum --dc 192.168.1.1 -d domain.com users.txt

In [ ]:
# Request a Kerberos Ticket (using impacket):

python3 GetNPUsers.py domain.com/ -usersfile users.txt -no-pass

In [ ]:
## DNS Enumeration

In [ ]:
# Query DNS for Domain Records (using dig):

dig @192.168.1.1 domain.com ANY

In [ ]:
# Perform a Zone Transfer (if allowed):

dig axfr @192.168.1.1 domain.com

In [ ]:
# Enumerate DNS Records (using dnsenum):

dnsenum domain.com

## Password Spraying

In [ ]:
# Perform a Password Spray Attack (using crackmapexec):

crackmapexec smb 192.168.1.1 -u users.txt -p passwords.txt

In [ ]:
# BloodHound Data Collection (Linux) Use SharpHound (via impacket or bloodhound-python):

bloodhound-python -d domain.com -u username -p password -ns 192.168.1.1 -c All


In [ ]:
# NTLM Authentication Testing: Check for NTLM Authentication (using ntlm-info):

ntlm-info -t 192.168.1.1

In [ ]:
# SNMP Enumeration: Enumerate SNMP Information (using snmpwalk):

snmpwalk -c public -v1 192.168.1.1

In [ ]:
# Vulnerability Scanning: Scan for Vulnerabilities (using nikto or nmap):

nikto -h 192.168.1.1
nmap --script vuln 192.168.1.1

# END TIME

In [ ]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)

# Parsing Script & Excel Deliverable Serialization

In [ ]:
#REQ: import requirements: pandas, openpyxl
import pandas as pd
import os
import csv
import glob
import xlsxwriter
import openpyxl


#path to parse to and read files from
path = "/root/.jupyter/{}/".format(FOLDER_NAME)

#all files ending in .csv
all_files = glob.glob(os.path.join(path, "*.csv"))

#initialize writer
writer = pd.ExcelWriter('/root/.jupyter/' + FOLDER_NAME + '/' + FOLDER_NAME + '_OSINT.xlsx', engine='openpyxl')

#write all files into excel from dataframes and name worksheet by filename 
print("The following files are being parsed to " + path + ":") 
print("")
for f in all_files:
    df = pd.read_csv(f)
    print(f)
    df.to_excel(writer, sheet_name=os.path.basename(f))

writer.save()  

print("")
print("Parsing of " + FOLDER_NAME + "_OSINT.xlsx Complete")

# Use Deepseek to review results

In [ ]:
import requests

# Define the API endpoint (replace with DeepSeek's actual API endpoint)
api_url = "https://api.deepseek.com/review"

# Prepare the payload (send the nmap output)
payload = {
    "output": nmap_output
}

# Send the request to the API
response = requests.post(api_url, json=payload)

# Check if the request was successful
if response.status_code == 200:
    review_results = response.json()
    print("DeepSeek Review Results:")
    print(review_results)
else:
    print("Error sending request to DeepSeek API:", response.status_code)

# Reference

- Recon: https://chat.deepseek.com/a/chat/s/5d783f90-4e9a-4af1-9ac9-538eeec422d0
- GOAD Walk thru: https://mayfly277.github.io/posts/GOADv2-pwning-part2/
- The Bear Necessities: https://spellzed.github.io/spellzedblog/2025/03/03/the-bear-necessities.html#lab-environment
- Game Of Active Directory, Learn how to attack and detect AD attacks: https://www.youtube.com/watch?v=XjgxQ6gHfyU
- DEF CON 32 - Winning the Game of Active Directory - Brandon Colley: https://www.youtube.com/watch?v=M-2d3sM3I2o&t=79s
- A Drop of Jupyter: A Modular Approach to Penetration Testing: https://www.youtube.com/watch?v=LTNKMA65BtI
- https://github.com/obheda12/JupyterPen
- https://github.com/obheda12/OSINT-Threat-Intel-Scripts